In [3]:
%pip install git+https://github.com/keras-team/keras-cv


  Cloning https://github.com/keras-team/keras-cv to /tmp/pip-req-build-389_12ey
  Running command git clone --filter=blob:none --quiet https://github.com/keras-team/keras-cv /tmp/pip-req-build-389_12ey
  Resolved https://github.com/keras-team/keras-cv to commit 94b0a551d03ee9de3d81663e2c1a680cb113f7f1
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 3.5 MB/s eta 0:00:00a 0:00:01
  Created wheel for keras-cv: filename=keras_cv-0.10.0-py3-none-any.whl size=975496 sha256=ac06ad6bd52c3da33ead782f95048971d232da81a1a1672049ea4b491b1701cd
  Stored in directory: /tmp/pip-ephem-wheel-cache-xtetx7gh/wheels/22/4a/22/95891344ab2777b97e94694fd4a187de0b50b831e9027b0f56
Successfully built keras-cv
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras_cv
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

In [21]:
# Load the CSV file
data_path = '/kaggle/input/ham10000-preprocessed-dataset/balanced and standardized images meta for custom.csv'  # Change this to your CSV file path
image_folder = '/kaggle/input/ham10000-preprocessed-dataset/balanced_and_standardized_images/balanced_and_standardized_images'  # Change this to your image folder path
df = pd.read_csv(data_path)

# Define parameters
IMG_SIZE = 96
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
DROPOUT_RATE = 0.5
EPOCHS = 20

# Get unique classes and create label mapping
classes = df['dx'].unique()
class_to_idx = {c: i for i, c in enumerate(classes)}
df['label'] = df['dx'].map(class_to_idx)

# Extract labels and encode them
label_encoder = LabelEncoder()
df["label"] = label_encoder.fit_transform(df["dx"])  # Encode the 'dx' column

In [22]:
# Train-test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

In [23]:
# Image Data Generators with Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 20% of train set will be used as validation
)

train_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='training'
)

val_generator = train_datagen.flow_from_dataframe(
    train_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', subset='validation'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    test_df, directory=image_folder, x_col='image_id', y_col='dx',
    target_size=(IMG_SIZE, IMG_SIZE), batch_size=BATCH_SIZE,
    class_mode='categorical', shuffle=False
)

Found 30039 validated image filenames belonging to 7 classes.
Found 7509 validated image filenames belonging to 7 classes.
Found 9387 validated image filenames belonging to 7 classes.


In [24]:
from tensorflow.keras import layers, models, backend as K

def shuffle_unit(inputs, out_channels, bottleneck_ratio=1, stride=1):
    in_channels = inputs.shape[-1]

    if stride == 1:
        x1 = layers.Lambda(lambda z: z[:, :, :, :in_channels // 2])(inputs)
        x2 = layers.Lambda(lambda z: z[:, :, :, in_channels // 2:])(inputs)
        shortcut = x1
        x = x2
    else:
        shortcut = inputs
        x = inputs

    x = layers.Conv2D(out_channels // 2, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.DepthwiseConv2D(3, strides=stride, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(out_channels // 2, 1, padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    if stride == 1:
        out = layers.Concatenate()([shortcut, x])
    else:
        shortcut = layers.AveragePooling2D(pool_size=3, strides=2, padding='same')(shortcut)
        out = layers.Concatenate()([shortcut, x])

    return out

def ShuffleNetV2(input_shape=(96, 96, 3), num_classes=10):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv2D(24, 3, strides=2, padding='same', use_bias=False)(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # Shuffle units
    for _ in range(3):
        x = shuffle_unit(x, 48)
    for _ in range(7):
        x = shuffle_unit(x, 96, stride=2)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, x)


model = ShuffleNetV2(input_shape=(96, 96, 3), num_classes=len(label_encoder.classes_))
model.compile(optimizer=Adam(learning_rate=1e-4), loss="categorical_crossentropy", metrics=["accuracy"])


In [25]:
# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]
)

Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


939/939 ━━━━━━━━━━━━━━━━━━━━ 549s 552ms/step - accuracy: 0.2442 - loss: 1.8797 - val_accuracy: 0.3259 - val_loss: 1.6744
Epoch 2/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 368s 390ms/step - accuracy: 0.3611 - loss: 1.5921 - val_accuracy: 0.4055 - val_loss: 1.4804
Epoch 3/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 345s 366ms/step - accuracy: 0.4140 - loss: 1.4780 - val_accuracy: 0.4312 - val_loss: 1.4418
Epoch 4/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 353s 373ms/step - accuracy: 0.4474 - loss: 1.3953 - val_accuracy: 0.4648 - val_loss: 1.3475
Epoch 5/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 331s 351ms/step - accuracy: 0.4683 - loss: 1.3398 - val_accuracy: 0.4838 - val_loss: 1.2909
Epoch 6/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 355s 376ms/step - accuracy: 0.4866 - loss: 1.2930 - val_accuracy: 0.4732 - val_loss: 1.3250
Epoch 7/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 328s 348ms/step - accuracy: 0.5038 - loss: 1.2588 - val_accuracy: 0.4891 - val_loss: 1.2989
Epoch 8/20
939/939 ━━━━━━━━━━━━━━━━━━━━ 327s 347ms/step - accuracy: 0.5146 - loss: 1.21

In [26]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns

def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

In [28]:
plot_model_training_curve(history)

In [29]:
# Test function
def evaluate_model():
    test_loss, test_acc = model.evaluate(test_generator)
    print(f'\nTest Accuracy: {test_acc * 100:.2f}%')

# Run evaluation
evaluate_model()

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning:

Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.



294/294 ━━━━━━━━━━━━━━━━━━━━ 110s 374ms/step - accuracy: 0.5799 - loss: 1.2222

Test Accuracy: 58.24%


In [30]:
model.save('/kaggle/working/shufflenet_model.keras') 

In [31]:
model.save_weights('/kaggle/working/shufflenet_model_weights.weights.h5') 